<a href="https://colab.research.google.com/github/jamieoliver/us-patents-240918/blob/main/us-patents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# US Patent Phrase to Phrase Matching

Based on https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners.

## Plan
- [x] Initial Setup
  - [x] Install relevant libraries
- [x] Prepare Datasets
  - [x] Download
  - [x] Tokenise
  - [x] Split into:
    - [x] Training
    - [x] Validation
    - [x] Test
- [ ] Train Model
- [ ] Test Model

## Initial Setup

In [1]:
!pip install -Uqq huggingface_hub datasets transformers pyarrow==15.0.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 15.0.2 which is incompatible.


## Prepare Datasets

### Download

In [2]:
from huggingface_hub import *

training_path = snapshot_download('jamieoliver/us-patents-240918',
                                  repo_type='dataset',
                                  allow_patterns='*.csv')

test_path = snapshot_download('jamieoliver/us-patents-test-240918',
                              repo_type='dataset',
                              allow_patterns='*.csv')

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

train.csv:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

test.csv:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

In [3]:
import pandas as pd
import os.path

training_data = pd.read_csv(os.path.join(training_path, 'train.csv'))
training_data

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [4]:
test_data = pd.read_csv(os.path.join(test_path, 'test.csv'))
test_data

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04
5,474c874d0c07bd21,dry corn,dry corn starch,C12
6,442c114ed5c4e3c9,tunneling capacitor,capacitor housing,G11
7,b8ae62ea5e1d8bdb,angular contact bearing,contact therapy radiation,B23
8,faaddaf8fcba8a3f,produce liquid hydrocarbons,produce a treated stream,C10
9,ae0262c02566d2ce,diesel fuel tank,diesel fuel tanks,F02


### Tokenise

In [5]:
def add_input(data):
  data['input'] = 'TEXT1: ' + data.context + '; TEXT2: ' + data.target + '; ANC1: ' + data.anchor

add_input(training_data)
training_data.input.head()

,input
0,TEXT1: A47; TEXT2: abatement of pollution; ANC...
1,TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2,TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3,TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4,TEXT1: A47; TEXT2: forest region; ANC1: abatement


In [6]:
add_input(test_data)
test_data.input.head()

,input
0,TEXT1: G02; TEXT2: inorganic photoconductor dr...
1,TEXT1: F23; TEXT2: altering gas flow; ANC1: ad...
2,TEXT1: B60; TEXT2: lower locating; ANC1: lower...
3,TEXT1: D06; TEXT2: upper portion; ANC1: cap co...
4,TEXT1: H04; TEXT2: artificial neural network; ...


In [7]:
from datasets import *

training_dataset = Dataset.from_pandas(training_data)
training_dataset

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [8]:
test_dataset = Dataset.from_pandas(test_data)
test_dataset

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'input'],
    num_rows: 36
})

In [9]:
from transformers import *

logging.set_verbosity_error()

tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
list(tokenizer.vocab.items())[:5]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
GroupViT models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version.Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
TAPAS models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version. Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


[('affey', 101253),
 ('▁Metformin', 88606),
 ('▁Delph', 107957),
 ('▁pjesë', 118355),
 ('▁wily', 84186)]

In [10]:
def tokenize(x):
  return tokenizer(x['input'])

training_dataset = training_dataset.map(tokenize, batched=True)
training_dataset[0]['input_ids']

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

[1,
 54453,
 435,
 294,
 336,
 5753,
 346,
 54453,
 445,
 294,
 47284,
 265,
 6435,
 346,
 23702,
 435,
 294,
 47284,
 2]

In [11]:
test_dataset = test_dataset.map(tokenize, batched=True)
test_dataset[0]['input_ids']

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

[1,
 54453,
 435,
 294,
 1098,
 4159,
 346,
 54453,
 445,
 294,
 31553,
 1456,
 48133,
 8263,
 346,
 23702,
 435,
 294,
 8847,
 1207,
 8263,
 2]

### Split

In [12]:
training_dataset, validation_dataset = training_dataset.train_test_split(0.25, seed=42).values()
dataset_dict = DatasetDict({'train':training_dataset,
                            'valid':validation_dataset,
                            'test':test_dataset})

dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    valid: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 36
    })
})